#### Notes:

##### Reference Scripts:
- https://github.com/ranayalcink/selenium_web_scraper/blob/main/glassdoor_scraper.py
- https://github.com/arapfaik/scraping-glassdoor-selenium/blob/master/glassdoor%20scraping.ipynb
- https://github.com/Khushi130404/Job-Scraper/blob/main/Multi_Glassdoor/Scrape_Glassdoor2.py
  
##### This script uses ChromeDriver to scrape. You can read how to download and get it set up below:

- https://developer.chrome.com/docs/chromedriver/get-started
- https://developer.chrome.com/docs/chromedriver/downloads 

In [6]:
#I copy and pasted this from the website above
#You can run this to ensure you've properly set up Chrome Driver

import time
from selenium import webdriver

driver = webdriver.Chrome('/path/to/chromedriver')  # Optional argument, if not specified will search path.
driver.get('http://www.google.com/')
time.sleep(5) # Let the user actually see something!
search_box = driver.find_element_by_name('q')
search_box.send_keys('ChromeDriver')
search_box.submit()
time.sleep(5) # Let the user actually see something!
driver.quit()

AttributeError: 'str' object has no attribute 'capabilities'

***
## Glassdoor Job Listings Scraper:

In [7]:
#Run this once to install necessary packages

!pip install webdriver-manager
!pip install pandas

In [4]:
#Load necessary packages:
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time, random, csv
import pandas as pd

In [3]:
def fetch_jobs(keyword, num_jobs, location):
    options = Options()
    options.add_argument("window-size=1920,1080")
    # Initialize the driver (ChromeDriverManager installs/upgrades the driver automatically)
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    
    # Open Glassdoor's main job search page
    driver.get("https://www.glassdoor.co.uk/Job/index.htm")
    time.sleep(random.uniform(1, 2))
    #close_popups() # Check for pop-ups after page loads
    
    # Enter search criteria (make sure these IDs are up-to-date)
    search_input = driver.find_element(By.ID, "searchBar-jobTitle")
    location_input = driver.find_element(By.ID, "searchBar-location")
    location_input.clear()
    search_input.send_keys(keyword)
    location_input.send_keys(location)
    location_input.send_keys(Keys.RETURN)
    time.sleep(random.uniform(1, 2))
    
    all_data = []  # List to store job data

    def random_sleep():
        time.sleep(random.uniform(1, 3))
        
    def click_load_more():
        try:
            load_more_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "button[data-test='load-more']"))
            )
            load_more_button.click()
            time.sleep(random.uniform(1, 2))  # wait for new jobs to load
            print("Clicked 'Load More' button")
            return True
        except Exception:
            print("No 'Load More' button found or error occurred")
            return False

    def close_popups():
        """Fast popup closer - only waits if popup is immediately visible"""
        popup_selectors = [
            "button.CloseButton[type='button']",  # Your existing selector
            "[data-test='close-icon']",           # Common close button
            "button[aria-label='Close']",         # Aria labeled close buttons
            ".modal-close",                       # Generic modal close
            "[data-test='sign-up-modal'] button", # Signup modal close
            ".close-button",                      # Generic close button class
            "button[title='Close']"               # Title attribute close
        ]
        
        for selector in popup_selectors:
            try:
                # Use find_elements (no wait) instead of WebDriverWait
                popups = driver.find_elements(By.CSS_SELECTOR, selector)
                if popups and popups[0].is_displayed():
                    popups[0].click()
                    print(f"Closed popup using selector: {selector}")
                    time.sleep(0.2)  # Much shorter sleep
                    return True
            except Exception:
                continue
        
        return False  # No popup found
    
    def safe_action_with_popup_check(action_func, *args, **kwargs):
        """Wrapper function that performs an action and checks for popups"""
        try:
            result = action_func(*args, **kwargs)
            close_popups()  # Check for popups after any action
            return result
        except Exception as e:
            close_popups()  # Check for popups if action fails
            raise e


    def scrape_page():
        nonlocal all_data
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        
        # Select all job cards – update the selector if necessary
        job_cards = soup.select("a[data-test='job-link']")
        jobs_scraped_this_page = 0
        
        for job_card in job_cards:
            # Check if we've reached the target number of jobs
            if len(all_data) >= num_jobs:
                print(f"Reached target of {num_jobs} jobs. Stopping scrape.")
                return jobs_scraped_this_page
            try:
                job_link = job_card.get('href')
                if job_link and job_link.startswith("/"):
                    job_link = "https://www.glassdoor.co.uk" + job_link
                print(f"Visiting job link: {job_link}")
                safe_action_with_popup_check(driver.get, job_link)
                close_popups() # Check for and close pop-ups
                random_sleep()
                
                # Attempt to click the "Show more" button to reveal full job description
                try:
                    show_more_button = driver.find_element(By.CSS_SELECTOR, '[data-test="show-more-cta"]')
                    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", show_more_button)
                    time.sleep(random.uniform(1, 2))
                    driver.execute_script("arguments[0].click();", show_more_button)
                    print("Clicked 'Show more' button successfully.")
                except Exception:
                    print("No 'Show more' button found.")
                    
                #Get the updated page source after clicking "Show more"
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                
                #Job Title
                title = soup.select_one("h1.heading_Heading__BqX5J")
                title = title.get_text(strip=True) if title else "No Title Available"

                #Company Name
                company = soup.select_one("h4.heading_Heading__BqX5J")
                company = company.get_text(strip=True) if company else "No Company Name Available"

                #Job Location
                location_elem = soup.select_one("[data-test='location']")
                location_text = location_elem.get_text(strip=True) if location_elem else "No Location Available"

                #Salary Estimate
                salary_elem = soup.select_one("[data-test='detailSalary']")
                salary_text = salary_elem.get_text(strip=True) if salary_elem else "Not Disclosed"
             
                # Extract all text from the job description
                job_desc_div = soup.find("div", class_="JobDetails_jobDescription__uW_fK JobDetails_showHidden__C_FOA")
                job_description = job_desc_div.get_text(strip=True) if job_desc_div else "No job description available"

                #Extract company overview details into own dictionary
                company_info = {}
                
                for item in soup.select(".JobDetails_overviewItem__cAsry"):
                    label_elem = item.select_one(".JobDetails_overviewItemLabel__KjFln")
                    value_elem = item.select_one(".JobDetails_overviewItemValue__xn8EF")
                
                    label_text = label_elem.get_text(strip=True) if label_elem else None
                    value_text = value_elem.get_text(strip=True) if value_elem else "No Data Available"
                
                    if label_text:
                        company_info[label_text] = value_text  # Stores in dictionary

                job_data = {
                    "Job Title": title,
                    "Company Name": company,
                    "Location": location_text,
                    "Salary": salary_text,
                    "Job Description": job_description,
                }
                
                # Merge company info dictionary into job_data
                job_data.update(company_info)

                # Append to final dataset
                all_data.append(job_data)
                jobs_scraped_this_page += 1
                print(f"Scraped job {len(all_data)}/{num_jobs}: {title}")
                
                driver.back()  # Return to the results list
                time.sleep(random.uniform(1, 2))
            except Exception as e:
                print(f"Error processing job card: {e}")
                driver.back()
                time.sleep(random.uniform(1, 2))
        
        return jobs_scraped_this_page

    # Iterate until we reach the target number of jobs
    current_page = 1
    try:
        while len(all_data) < num_jobs:
            print(f"Scraping page {current_page} (Jobs collected so far: {len(all_data)}/{num_jobs})")
    
            jobs_scraped = scrape_page()
    
            if len(all_data) >= num_jobs:
                break
    
            if jobs_scraped == 0:
                load_more_clicked = False
                while click_load_more() and len(all_data) < num_jobs:
                    jobs_scraped = scrape_page()
                    close_popups()
                    load_more_clicked = True
                    if len(all_data) >= num_jobs:
                        break
    
                if not load_more_clicked or jobs_scraped == 0:
                    try:
                        next_page_button = driver.find_element(By.CSS_SELECTOR, ".button_Button__MlD2g.button-base_Button__knLaX")
                        next_page_button.click()
                        time.sleep(random.uniform(1, 2))
                        current_page += 1
                    except Exception as e:
                        print(f"Could not navigate to next page and no more jobs found: {e}")
                        break
            else:
                while click_load_more() and len(all_data) < num_jobs:
                    scrape_page()
                    if len(all_data) >= num_jobs:
                        break
    
                if len(all_data) < num_jobs:
                    try:
                        next_page_button = driver.find_element(By.CSS_SELECTOR, ".button_Button__MlD2g.button-base_Button__knLaX")
                        next_page_button.click()
                        close_popups()
                        time.sleep(random.uniform(1, 2))
                        current_page += 1
                    except Exception as e:
                        print(f"Could not navigate to next page: {e}")
                        break
    except KeyboardInterrupt:
        print("Scraping interrupted by user.")
    finally:
        driver.quit()
        
        if all_data:
            clean_keyword = keyword.replace(' ', '-').replace('/', '-').replace('\\', '-')
            clean_location = location.replace(' ', '-').replace('/', '-').replace('\\', '-')
            csv_file = f"GDJobsScraper_{clean_keyword}_{clean_location}.csv"
    
            keys = all_data[0].keys()
            try:
                with open(csv_file, "w", newline="", encoding="utf-8") as output_file:
                    dict_writer = csv.DictWriter(output_file, fieldnames=keys)
                    dict_writer.writeheader()
                    dict_writer.writerows(all_data)
                print(f"Scraping complete. Data saved to {csv_file}.")
                print(f"Total jobs scraped: {len(all_data)}")
            except Exception as e:
                print(f"Error saving CSV file: {e}")
        else:
            print("No job data was scraped.")

In [5]:
try:
    fetch_jobs("economist", 5000, "United Kingdom")
except KeyboardInterrupt:
    print("Notebook kernel interrupted.")
except Exception as e:
    print(f"Unexpected error: {e}")

Scraping page 1 (Jobs collected so far: 0/5000)
Visiting job link: https://www.glassdoor.co.uk/partner/jobListing.htm?pos=101&ao=1110586&s=58&guid=000001975917d7d6a5761c354f43879e&src=GD_JOB_AD&t=SR&vt=w&ea=1&cs=1_84ed8831&cb=1749546424652&jobListingId=1009768961093&cpc=3BA4CE39D5B5DEF5&jrtk=5-yul1-0-1itchfm13gbnl800-6460a6fe32b12c3a---6NYlbfkN0CC3HtZmsoU95stywJrJs6Umyt8PRsUTN6M2TTgTd8kQf86XrIo6E__V3u0vF8sGZfotESjbMFWcqvVFV8UEfL5Q1QyWbt07QcZ9Baj3Nj4xTIXrhkY9yV4L2aXVaiiD5-iox1MSL1q4tvmQ9O9Q70uNzftmjN5XCgy0oKxb1upi_9Vv3uKD6ZWTNtANdByvpg_YcSpkWfa8CEDY-p-PRzbm1o9fMFeQYo-EykxVdFCYVuGIXVMXkE4u51zn6jhtzzMusFwBZJ9LV9abP-AGnsaHjnSGl-IcRrPXZaiiR0Y3xbJt-Gk9mf01jbw63hdWGrPfORveLlld-v0mJnJJ72b8HVb4U_NRokHB0wCYQMwuSUyJ-IVqOUmTtT97EjLnawhYgC3PWl0D6KcelD8cXYW7QOIxrKk440hgzt0lGFAZdsPVuqV-A4EDb1bcM4g20reGu8WFwMZTd76a8ZJt8_hO_2cOh4ghmNDAII9c5v53AGTAF7SK688TLhJHnIeuj96Z5mg9xNGho1oYQKf3GZQb-Xt8Ki4AuMUY8JSnoKjVZmFwPq-aiw_voZk-7Lxkyjp-17W32V5XbW22qUZi4pwSO017hJS0HPUO6VGgvf_-IFjTnZVTNFtMA32uuXRyIR-zjyiEdLzZFm